In [51]:
import os
import numpy as np
import ROOT
from pprint import pprint

In [59]:
ERA = "2018"
MASSPOINT = "MHc-130_MA-90"
CHANNEL = "Skim3Mu"
NETWORK = "GraphNet"

DataStream = "DoubleMuon"
conv        = ["DYJets", "ZGToLLG"]
VV          = ["WZTo3LNu_amcatnlo","ZZTo4L_powheg"]
ttX         = ["ttWToLNu", "ttZToLLNuNu", "ttHToNonbb"]
rare        = ["WWW", "WWZ", "WZZ", "ZZZ","WWG", "TTG", "TTTT", "tZq", "tHq", "VBF_HToZZTo4L", "GluGluHToZZTo4L"]
MCSamples   = conv + VV + ttX + rare

MA = 90.
SIGMA = 0.743059
WIDTH = 1.15738
WINDOW = np.sqrt(np.power(SIGMA, 2)+np.power(WIDTH, 2))
print(WINDOW)

WORKDIR = "/home/choij/workspace/ChargedHiggsAnalysis"

1.3753781814035733


In [60]:
def getEventRates(fstring: str, histkey: str) -> float:
    f = ROOT.TFile.Open(fstring)
    # Get event rates
    histkey = f"SR3Mu/Central/{MASSPOINT}/3D"
    h = f.Get(histkey); h.SetDirectory(0)
    f.Close()

    xL = h.GetXaxis().FindBin(MA-WINDOW)
    xR = h.GetXaxis().FindBin(MA+WINDOW)
    h.GetXaxis().SetRange(xL, xR)

    return h.Integral()

In [65]:
RATEs = {}
# observation
fstring = f"{WORKDIR}/data/PromptEstimator/{ERA}/{CHANNEL}__{NETWORK}__/DATA/PromptEstimator_SkimTree_SS2lOR3l_{DataStream}.root"
histkey = f"SR3Mu/Central/{MASSPOINT}/3D"
assert os.path.exists(fstring)
RATEs["observation"] = getEventRates(fstring, histkey)

# signal
fstring = f"{WORKDIR}/data/PromptEstimator/{ERA}/{CHANNEL}__{NETWORK}__WeightVar__/PromptEstimator_TTToHcToWAToMuMu_{MASSPOINT}.root"
histkey = f"SR3Mu/Central/{MASSPOINT}/3D"
assert os.path.exists(fstring)
RATEs["signal"] = getEventRates(fstring, histkey)

# nonprompt
fstring = f"{WORKDIR}/data/NonpromptEstimator/{ERA}/{CHANNEL}__{NETWORK}__/DATA/NonpromptEstimator_SkimTree_SS2lOR3l_DoubleMuon.root"
histkey = f"SR3Mu/Central/{MASSPOINT}/3D"
assert os.path.exists(fstring)
RATEs["nonprompt"] = getEventRates(fstring, histkey)

# diboson
rate = 0.
for sample in VV:
    fstring = f"{WORKDIR}/data/PromptEstimator/{ERA}/{CHANNEL}__{NETWORK}__WeightVar__/PromptEstimator_SkimTree_SS2lOR3l_{sample}.root" 
    histkey = f"SR3Mu/Central/{MASSPOINT}/3D"
    assert os.path.exists(fstring)
    rate += max(0., getEventRates(fstring, histkey))
RATEs["diboson"] = rate

# conversion
rate = 0.
for sample in conv:
    fstring = f"{WORKDIR}/data/PromptEstimator/{ERA}/{CHANNEL}__{NETWORK}__WeightVar__/PromptEstimator_SkimTree_SS2lOR3l_{sample}.root" 
    histkey = f"SR3Mu/Central/{MASSPOINT}/3D"
    assert os.path.exists(fstring)
    rate += max(0., getEventRates(fstring, histkey))
RATEs["conversion"] = rate

# ttX
rate = 0.
for sample in ttX:
    fstring = f"{WORKDIR}/data/PromptEstimator/{ERA}/{CHANNEL}__{NETWORK}__WeightVar__/PromptEstimator_SkimTree_SS2lOR3l_{sample}.root" 
    histkey = f"SR3Mu/Central/{MASSPOINT}/3D"
    assert os.path.exists(fstring)
    rate += max(0., getEventRates(fstring, histkey))
RATEs["ttX"] = rate

# rare
rate = 0.
for sample in rare:
    fstring = f"{WORKDIR}/data/PromptEstimator/{ERA}/{CHANNEL}__{NETWORK}__WeightVar__/PromptEstimator_SkimTree_SS2lOR3l_{sample}.root" 
    if not os.path.exists(fstring):
         fstring = f"{WORKDIR}/data/PromptEstimator/{ERA}/{CHANNEL}__{NETWORK}__WeightVar__/PromptEstimator_{sample}.root" 
    histkey = f"SR3Mu/Central/{MASSPOINT}/3D"
    assert os.path.exists(fstring)
    rate += max(0., getEventRates(fstring, histkey))
RATEs["rare"] = rate

pprint(RATEs)

{'conversion': 0.34807202983737073,
 'diboson': 48.88774773224065,
 'nonprompt': 70.04833576705288,
 'observation': 186.0,
 'rare': 17.797327006892978,
 'signal': 86.64258815794454,
 'ttX': 43.18042173471089}


In [66]:
print("imax\t1 number of bins")
print("jmax\t5 number of processes minus 1")
print("kmax\t* number of nuisance parameters")
print("--------------------------------------------------------------------------")
print("--------------------------------------------------------------------------")
print("bin\tsignal_region")
print(f"observation\t{RATEs['observation']}")
print("bin\tsignal_region\tsignal_region\tsignal_region\tsignal_region\tsignal_region\tsignal_region")
print("process\tnonprompt\tdiboson\tconversion\ttt+X\trare\tsignal")
print("process\t1\t2\t3\t4\t5\t0")
print(f"rate\t{RATEs['nonprompt']}\t{RATEs['diboson']}\t{RATEs['conversion']}\t{RATEs['ttX']}\t{RATEs['rare']}\t{RATEs['signal']}")
print("--------------------------------------------------------------------------")

imax	1 number of bins
jmax	5 number of processes minus 1
kmax	* number of nuisance parameters
--------------------------------------------------------------------------
--------------------------------------------------------------------------
bin	signal_region
observation	186.0
bin	signal_region	signal_region	signal_region	signal_region	signal_region	signal_region
process	nonprompt	diboson	conversion	tt+X	rare	signal
process	1	2	3	4	5	0
rate	70.04833576705288	48.88774773224065	0.34807202983737073	43.18042173471089	17.797327006892978	86.64258815794454
--------------------------------------------------------------------------
